Libraries

In [21]:
import tensorflow as tf
import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

# Data Preprocessing

Preprocessing the Training Set

In [22]:
# Transform the Training Set to avoid overfitting
training_generator = ImageDataGenerator(rescale = 1./255,
                                        shear_range = 0.2,
                                        zoom_range = 0.2,
                                        horizontal_flip = True)

In [23]:
training_set = training_generator.flow_from_directory("dataset/training_set",
                                                      target_size = (64, 64),
                                                      batch_size = 32,
                                                      class_mode = "binary")

Found 8048 images belonging to 2 classes.


Preprocessing the Test Set

In [24]:
# Only scale the images as the training set
test_generator = ImageDataGenerator(rescale = 1./255)

In [25]:
# Set the target size just like the training set
test_set = test_generator.flow_from_directory("dataset/test_set",
                                              target_size = (64, 64),
                                              batch_size = 32,
                                              class_mode = "binary")

Found 2000 images belonging to 2 classes.


# Convolutional Neural Networks

Initializing the CNN

In [26]:
convolutional = tf.keras.models.Sequential()

Convolution

In [27]:
# Kernel Size -> The size of the feature detector, n x n matrices
# Input Shape -> (Target size, rgb: B&W = 1, colors = 3)
convolutional.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = "relu", input_shape = (64, 64, 3)))

Pooling

In [28]:
# Strides -> The Changes of each iteration
# Pool size -> The frame in each iteration, n x n matrices
# Padding -> To ignore cells that has no complete 2x2 form of matrices
convolutional.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

Second Convolutional Layer

In [29]:
convolutional.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = "relu"))
convolutional.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

Flattening

In [30]:
convolutional.add(tf.keras.layers.Flatten())

Complete Connection

In [31]:
# Fully Connected Graph of Neurons
convolutional.add(tf.keras.layers.Dense(units = 128, activation = "relu", ))

Output Layer

In [32]:
# Set the units as 1 because it is a binary classification
# Relu is bad to use as the activation function of the output layer
# If Binary Classification -> Sigmoid Function
# If Multiclass Classification -> Softmax Function
convolutional.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Training the Convolutional Neural Networks

Compiling the CNN

In [33]:
convolutional.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

Training the CNN and Validating the CNN with Test Set

In [34]:
convolutional.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
252/252 [==============================] - 29s 112ms/step - loss: 0.6883 - accuracy: 0.5484 - val_loss: 0.6160 - val_accuracy: 0.6705
Epoch 2/25
252/252 [==============================] - 28s 109ms/step - loss: 0.6212 - accuracy: 0.6554 - val_loss: 0.5851 - val_accuracy: 0.6915
Epoch 3/25
252/252 [==============================] - 26s 104ms/step - loss: 0.5689 - accuracy: 0.7030 - val_loss: 0.6318 - val_accuracy: 0.6600
Epoch 4/25
252/252 [==============================] - 26s 105ms/step - loss: 0.5428 - accuracy: 0.7259 - val_loss: 0.5541 - val_accuracy: 0.7255
Epoch 5/25
252/252 [==============================] - 26s 104ms/step - loss: 0.5016 - accuracy: 0.7494 - val_loss: 0.4946 - val_accuracy: 0.7695
Epoch 6/25
252/252 [==============================] - 26s 105ms/step - loss: 0.4857 - accuracy: 0.7605 - val_loss: 0.5164 - val_accuracy: 0.7425
Epoch 7/25
252/252 [==============================] - 27s 107ms/step - loss: 0.4580 - accuracy: 0.7746 - val_loss: 0.4757 - val_ac

# Predict Single Data

Load the Image with Keras

In [35]:
test_image_1 = image.load_img("dataset/single_prediction/cat_or_dog_1.jpg", target_size = (64, 64))
test_image_2 = image.load_img("dataset/single_prediction/cat_or_dog_2.jpg", target_size = (64, 64))

Transform the Image to Array

In [36]:
test_image_1 = image.img_to_array(test_image_1)
test_image_2 = image.img_to_array(test_image_2)

Add Extra Dimension as the Batch in the Image

In [37]:
test_image_1 = np.expand_dims(test_image_1, axis = 0)
test_image_2 = np.expand_dims(test_image_2, axis = 0)

Check Each Class Indices of the Training

In [38]:
training_set.class_indices

{'cats': 0, 'dogs': 1}

Making New Predictions

In [39]:
prediction1 = convolutional.predict(test_image_1)

In [40]:
prediction2 = convolutional.predict(test_image_2)

Get the Result from the Batch

In [42]:
def check_pred_from_batch(prediction_batch):
  if(prediction_batch[0][0] == 1):
    print("Dog")
  else:
    print("Cat")

In [43]:
check_pred_from_batch(prediction1)

Dog


In [44]:
check_pred_from_batch(prediction2)

Cat
